In [38]:
'''
pairwise sequence alignment 
- linear gap penalty
- divide conquer

input A and B
output spscore, align_A, align_B
author: Juntao Chen
'''

class PSA_DC(object):
    """
    pairwise sequence alignment with divide conquer method

    Args:
        s: sequence 1
        t: sequence 2
        sa: aligned seq 1
        ta: aligned seq 2
    """
    def __init__(self, s, t):
        self.g = -1
        self.s = s
        self.t = t
        # record the match location in t or gap
        self.Align_s = ['']*len(s)
        self.sa, self.ta = self.Align(0, len(s), 0, len(t))
        
    # mathch 1 mismatch -1
    def p(self, i, j):
        return int(self.s[i]==self.t[j]) - int(self.s[i]!=self.t[j])

    def match(self, si, tj):
        return int(si==tj) - int(si!=tj)

    def BestScore(self, s1, t1):
        """
        compute the sp score of two seqs
        """
        m = len(s1)
        n = len(t1)
        a = [0] * (n+1)
        
        for j in range(0, n+1):
            a[j] = j * self.g

        for i in range(1, m+1):
            old = a[0]
            a[0] = i * self.g
            for j in range(1, n+1):
                temp = a[j]
                a[j] = max(a[j] + self.g, old + self.match(s1[i-1], t1[j-1]), a[j-1] + self.g)
                old = temp
        return a


    # Divide and conquer strategy
    def Align(self, a, b, c, d):
        s_align = list(self.s)
        t_align = list(self.t)

        if self.s[a:b] == '' or self.t[c:d] == '':
            if self.s[a:b]:
                self.Align_s[a:b] = ['-']*len(self.s[a:b])
            return 
        else:
            i = (a+b)//2
            # dimension: d-c
            pref_sim = self.BestScore(self.s[a:i], self.t[c:d])
            t_rev = self.t[c:d][::-1]
            s_rev = self.s[i+1:b][::-1]
            suff_sim = self.BestScore(s_rev, t_rev)
            suff_sim = suff_sim[::-1]

            posmax = c - 1
            typemax = '-'
            Vmax = pref_sim[0] + self.g + suff_sim[0]

            # find the best match (i,j) i from s, j from t
            for j in range(c, d):
                if pref_sim[j-c] + self.p(i,j) + suff_sim[j+1-c] > Vmax:
                    posmax = j
                    typemax = 'N'
                    Vmax = pref_sim[j-c] + self.p(i,j) + suff_sim[j-c+1]

                if pref_sim[j-c+1] + self.g + suff_sim[j-c+1] > Vmax:
                    posmax = j
                    typemax = '-'
                    Vmax = pref_sim[j-c] + self.g + suff_sim[j-c]
            # i match gap
            if typemax == '-':
                self.Align_s[i] = '-' 
                self.Align(a, i, c, posmax+1)
                self.Align(i+1, b, posmax+1, d)
            # i match j
            else:
                self.Align_s[i] = posmax
                self.Align(a, i, c, posmax)
                self.Align(i+1, b, posmax+1, d)
            # when record matrix finish the finding process
            # then trace back the best alignment
            if '' not in self.Align_s:
                j = 0
                k_t = 0
                k_s = 0
                for i in self.Align_s:
                    if i == '-':
                        t_align.insert(j+1+k_t, '-')
                        k_t += 1
                    elif i-j > 1:
                        if j not in self.Align_s:
                            j -= 1
                        for ii in range(i-j-1):
                            s_align.insert(ii+self.Align_s.index(i)+k_s,'-')
                        k_s += i-j-1
                        j = i
                    else:
                        j = i

        return ''.join(s_align), ''.join(t_align)

In [39]:
if __name__ == "__main__":
    # input sequences
    t = 'TTGCCATcasTTT'
    s = 'CCAATTTTTTASD'

    psa = PSA_DC(s, t)

    print(psa.sa)
    print(psa.ta)

---CCAATTT-TTTASD
TTGCC-ATcasTTT---
